# 延后初始化
:label:`sec_deferred_init`

到目前为止，我们忽略了建立网络时需要做的以下这些事情：

* 我们定义了网络架构，但没有指定输入维度。
* 我们添加层时没有指定前一层的输出维度。
* 我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。

有些读者可能会对我们的代码能运行感到惊讶。
毕竟，深度学习框架无法判断网络的输入维度是什么。
这里的诀窍是框架的*延后初始化*（defers initialization），
即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

在以后，当使用卷积神经网络时，
由于输入维度（即图像的分辨率）将影响每个后续层的维数，
有了该技术将更加方便。
现在我们在编写代码时无须知道维度是什么就可以设置参数，
这种能力可以大大简化定义和修改模型的任务。
接下来，我们将更深入地研究初始化机制。

## 实例化网络

首先，让我们实例化一个多层感知机。


此时，因为输入维数是未知的，所以网络不可能知道输入层权重的维数。
因此，框架尚未初始化任何参数，我们通过尝试访问以下参数进行确认。


In [5]:
# pytorch demo code
import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.Linear(256, 10))
print(net)

[net[i].state_dict() for i in range(len(net))]

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


[OrderedDict([('weight', <UninitializedParameter>),
              ('bias', <UninitializedParameter>)]),
 OrderedDict(),
 OrderedDict([('weight',
               tensor([[-0.0407,  0.0472,  0.0254,  ..., -0.0343,  0.0459, -0.0057],
                       [ 0.0479, -0.0403, -0.0185,  ...,  0.0287, -0.0492,  0.0041],
                       [ 0.0021, -0.0623,  0.0323,  ...,  0.0613,  0.0095, -0.0431],
                       ...,
                       [-0.0310, -0.0036,  0.0108,  ..., -0.0563,  0.0504,  0.0511],
                       [ 0.0237, -0.0206, -0.0154,  ...,  0.0423,  0.0211,  0.0345],
                       [ 0.0236,  0.0124,  0.0109,  ..., -0.0156, -0.0184,  0.0370]])),
              ('bias',
               tensor([ 0.0227,  0.0002, -0.0607, -0.0409, -0.0427, -0.0447,  0.0129, -0.0299,
                        0.0370,  0.0331]))])]

接下来让我们将数据通过网络，最终使框架初始化参数。


In [6]:
low = torch.finfo(torch.float32).min / 10
high = torch.finfo(torch.float32).max / 10
X = torch.zeros([2, 20], dtype=torch.float32).uniform_(low, high)
net(X)
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


一旦我们知道输入维数是20，框架可以通过代入值20来识别第一层权重矩阵的形状。
识别出第一层的形状后，框架处理第二层，依此类推，直到所有形状都已知为止。
注意，在这种情况下，只有第一层需要延迟初始化，但是框架仍是按顺序初始化的。
等到知道了所有的参数形状，框架就可以初始化参数。

## 小结

* 延后初始化使框架能够自动推断参数形状，使修改模型架构变得容易，避免了一些常见的错误。
* 我们可以通过模型传递数据，使框架最终初始化参数。

## 练习

1. 如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？
1. 如果指定了不匹配的维度会发生什么？
1. 如果输入具有不同的维度，需要做什么？提示：查看参数绑定的相关内容。


[Discussions](https://discuss.d2l.ai/t/5770)


In [7]:
# 可以正常运行。第一层会立即初始化, 但其他层同样是直到数据第一次通过模型传递才会初始化
net = nn.Sequential(
    nn.Linear(20, 256), nn.ReLU(),
    nn.LazyLinear(128), nn.ReLU(),
    nn.LazyLinear(10)
)
print(net[0].weight)
print(net[2].weight)
net(X)
print(net[2].weight)

Parameter containing:
tensor([[-0.0332,  0.2116,  0.1954,  ...,  0.0415, -0.0947, -0.1437],
        [-0.1176, -0.2035,  0.1023,  ..., -0.0310, -0.0680, -0.0816],
        [ 0.0878, -0.0812, -0.1928,  ...,  0.1078,  0.1349, -0.0884],
        ...,
        [ 0.1233,  0.1685,  0.1304,  ...,  0.0351,  0.1314,  0.0317],
        [-0.0122, -0.0439, -0.1655,  ...,  0.1987,  0.0303,  0.2225],
        [ 0.0383, -0.0610,  0.0531,  ...,  0.0708,  0.0167,  0.1286]],
       requires_grad=True)
<UninitializedParameter>
Parameter containing:
tensor([[ 0.0086,  0.0072, -0.0610,  ...,  0.0080, -0.0067, -0.0350],
        [-0.0443, -0.0530,  0.0069,  ..., -0.0342,  0.0613, -0.0308],
        [ 0.0030, -0.0541,  0.0474,  ...,  0.0015, -0.0341,  0.0187],
        ...,
        [ 0.0483,  0.0100, -0.0261,  ..., -0.0370, -0.0191,  0.0597],
        [-0.0181, -0.0469, -0.0067,  ..., -0.0248, -0.0174, -0.0395],
        [ 0.0293,  0.0091, -0.0461,  ..., -0.0533, -0.0532, -0.0430]],
       requires_grad=True)
